In [ ]:
import random

import IPython
import IPython.display

import numpy as np

import tensorflow as tf


from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
import pandas as pd

# TF CHECK

In [ ]:

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
# tf.test.is_built_with_cuda()
# tf.executing_eagerly()

# Case 

In [ ]:
folders = ['Rough','UTS','Elon']
target = ['roughness (μm)','tension_strength (MPa)','elongation (%)']
index = 2
CASE = folders[index]

In [ ]:
from prepross import importAndProcess
xtrain,xval,xtest,ytrain,yval,ytest = importAndProcess(index)
x = pd.concat((xtrain,xtest,xval),axis=0)
y = pd.concat((ytrain,ytest,yval),axis=0)

# Network

In [ ]:
# Hyperparameters
lossfn = [tf.keras.losses.MeanSquaredError(),tf.keras.losses.MeanAbsolutePercentageError(),tf.keras.losses.MeanAbsoluteError()]
lossfn = lossfn[index]
optimizer = [tf.keras.optimizers.Adam(learning_rate=i,beta_1=0.09,beta_2=0.9) for i in [5e-2,1e-2,5e-3]]
optimizer = optimizer[index]
metrics = ['mean_squared_error','mean_absolute_error',tf.keras.losses.MeanAbsolutePercentageError()]

In [ ]:
#Give a random seed
tf.random.set_seed(random.randint(1,1000))
from networks import create_model,compile_and_fit

In [ ]:
mArch = []
for l2 in range(2,11):
    for l3 in range(2,12):
        if l2*(len(xtrain.columns)+l3) + l3 < 100 and l2*(len(xtrain.columns)+l3) + l3 > 30:
            mArch.append([[l2,'sigmoid',1],[l3,'tanh',1]])
            mArch.append([[l2,'linear',1],[l3,'relu',1]])

In [ ]:
len(mArch)

In [ ]:
val_performance = {}
performance = {}

for ele in mArch:
    # Create a basic model Architecture
    modelArchitecture = [ele[0],ele[1],[1,'linear',1]]
    modelName = ''
    for u,fn,b in modelArchitecture:
        modelName += str(u)+fn+str(b)+'_'

    # Create Model
    model = create_model(modelArchitecture,modelName,len(xtrain.columns))
    # Display the model's architecture
    model.summary()

    history = compile_and_fit(CASE,model,optimizer,lossfn,metrics,
                            modelName,xtrain,ytrain,xval,yval,
                            verbose = 0,batch_size=3,MAX_EPOCHS=1000,
                            patience = 100,record = True)

    val_performance[modelName] = model.evaluate(xval,yval,verbose=1)
    performance[modelName] = model.evaluate(xtest,ytest, verbose=1)
    # print(val_performance[modelName])
    # print(performance[modelName])